In [2]:
%pip install --upgrade selenium
%pip install webdriver-manager
%pip urllib.parse
%pip install requests beautifulsoup4
%pip install pandas matplotlib seaborn
%pip install numpy scipy scikit-learn
%pip install selenium
%pip install tabulate


  Using cached selenium-4.26.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.27.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl.metadata (13 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached selenium-4.26.1-py3-none-any.whl (9.7 MB)
Using cached trio-0.27.0-py3-none-any.whl (481 kB)
Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
Using cached web

ERROR: unknown command "urllib.parse"



Note: you may need to restart the kernel to use updated packages.
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ----------------------- ---------------- 6.8/11.5 MB 33.7 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.5 MB 28.2 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 26.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ------------------ --------------------- 3.7/7.8 MB 19.2 MB/s eta 0:00:01
   ------------------------------ --------- 6.0/7.8 MB 15.3 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 12.8 MB/s eta 0:00:00


In [11]:
%pip install openpyxl
%pip install retrying

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import re

from pylab import rcParams
rcParams['figure.figsize'] = 7, 5
rcParams['font.size'] = 10
rcParams['axes.facecolor'] = 'white'
%matplotlib inline

#Data Acquisition
import requests                 #How Python gets the webpages
from bs4 import BeautifulSoup   #Creates structured, searchable object

import os
from tabulate import tabulate
from datetime import datetime, timedelta
import time


from selenium import webdriver #Provides an API for controlling web browsers programmatically.

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from urllib.parse import urlparse
from urllib.parse import urlparse, parse_qs
from retrying import retry
import re

from webdriver_manager.chrome import ChromeDriverManager



In [40]:
import logging
# 设置日志
def setup_logger(listing_id):
    # 从URL中提取listing_id
    if isinstance(listing_id, str) and 'rooms/' in listing_id:
        listing_id = listing_id.split('rooms/')[-1].split('?')[0]
    
    # 创建logs文件夹（如果不存在）
    if not os.path.exists('logs'):
        os.makedirs('logs')
    
    # 设置日志文件名，包含时间戳和listing_id
    log_filename = f'logs/airbnb_scraper_{listing_id}_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
    
    # 配置日志
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s [%(levelname)s] %(message)s',
        handlers=[
            logging.FileHandler(log_filename),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def select_dates(driver, checkin_date, checkout_date):
    logger.info(f"尝试选择日期: 入住 {checkin_date}, 退房 {checkout_date}")
    
    try:
        # 等待日历加载
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "_1ncwt1cd"))
        )
        
        # 查找并点击入住日期
        checkin_xpath = f"//td[@role='button'][@aria-label[contains(., '{checkin_date}')]]"
        checkin_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, checkin_xpath))
        )
        logger.info("找到入住日期元素")
        checkin_element.click()
        logger.info("已点击入住日期")
        
        # 等待一下确保日历状态更新
        time.sleep(1)
        
        # 查找并点击退房日期
        checkout_xpath = f"//td[@role='button'][@aria-label[contains(., '{checkout_date}')]]"
        checkout_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, checkout_xpath))
        )
        logger.info("找到退房日期元素")
        checkout_element.click()
        logger.info("已点击退房日期")
        
        return True
        
    except Exception as e:
        logger.error(f"选择日期失败: {str(e)}")
        return False

### Web Scraping

In [41]:
@retry(stop_max_attempt_number=3, wait_fixed=2000)


def scrape_listing_pricing(listing_url, guests, num_days):

 # 设置日志
    logger = setup_logger(listing_url)
    logger.info(f"开始抓取房源: {listing_url}")
    logger.info(f"设置: guests={guests}, num_days={num_days}")

    df = pd.DataFrame(columns=[
        'Check-in Date', 'Check-out Date', 'Host', 
        'Guest', 'bedrooms', 'beds', 'baths', 'years_hosting',
        'nightly_price', 'cleaning_fee', 'service_fee', 'total',
        'availability'
    ])
    
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    
    
    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        logger.info("Chrome WebDriver 初始化成功")

        current_date = datetime.now()
        end_date = current_date + timedelta(days=num_days)
        
        # 首先访问页面获取房屋基本信息
        url = f"{listing_url}"
        logger.info(f"访问页面: {url}")
        driver.get(url)
        time.sleep(5)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        logger.info("页面解析完成")
        
        # 获取房屋基本信息
        try:
            house_elements = soup.findAll(class_='l7n4lsf atm_9s_1o8liyq_keqd55 dir dir-ltr')
            logger.info(f"找到 {len(house_elements)} 个房屋信息元素")

            if house_elements and len(house_elements) >= 5:
                def safe_extract_number(text):
                    match = re.search(r'\d+', text)
                    return float(match.group()) if match else 'NaN'
                
                Guests = safe_extract_number(house_elements[0].text)
                bedrooms = safe_extract_number(house_elements[1].text)
                beds = safe_extract_number(house_elements[2].text)
                baths = safe_extract_number(house_elements[3].text)
                years_hosting = safe_extract_number(house_elements[4].text)

                logger.info(f"房屋信息: Guests={Guests}, bedrooms={bedrooms}, beds={beds}, "
                          f"baths={baths}, years_hosting={years_hosting}")
            else:
                logger.warning("未找到足够的房屋信息元素")
                Guests = bedrooms = beds = baths = years_hosting = 'NaN'
        except Exception as e:
            logger.error(f"获取房屋信息出错: {str(e)}")
            Guests = bedrooms = beds = baths = years_hosting = 'NaN'

        success_count = 0
        fail_count = 0
        
        while current_date <= end_date:
            try:
                checkin_date = current_date.strftime('%Y-%m-%d')
                checkout_date = (current_date + timedelta(days=2)).strftime('%Y-%m-%d')

                logger.info(f"\n处理日期: {checkin_date} 到 {checkout_date}")
                
                url = f"{listing_url}?check_in={checkin_date}&guests={guests}&adults={guests}&check_out={checkout_date}"
                driver.get(url)
                time.sleep(3)
                
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                # 检查日期可用性
                date_cells = soup.find_all('td', attrs={'role': 'button'})
                logger.info(f"找到 {len(date_cells)} 个日期单元格")
                is_available = True  # 默认可用
                
                for cell in date_cells:
                    date_label = cell.get('aria-label', '')
                    if checkin_date in date_label:
                        is_available = cell.get('aria-disabled') != 'true'
                        logger.info(f"日期 {checkin_date} 可用性: {'可用' if is_available else '不可用'}")

                        break
                
                # 如果日期不可用，添加基本信息并跳过价格抓取
                if not is_available:
                    logger.info(f"日期 {checkin_date} 不可预订，跳过价格抓取")
                    df = pd.concat([df, pd.DataFrame({
                        'Host': [listing_url],
                        'Check-in Date': [checkin_date],
                        'Check-out Date': [checkout_date],
                        'Guest': [Guests],
                        'bedrooms': [bedrooms],
                        'beds': [beds],
                        'baths': [baths],
                        'years_hosting': [years_hosting],
                        'nightly_price': ['NaN'],
                        'cleaning_fee': ['NaN'],
                        'service_fee': ['NaN'],
                        'total': ['NaN'],
                        'availability': ['Unavailable']
                    })], ignore_index=True)
                    print(f"{checkin_date} 不可预订，跳过价格抓取")
                    current_date += timedelta(days=1)
                    continue
                
                # 日期可用时抓取价格信息
                try:
                    scripts = soup.find_all('script', {'type': 'application/json'})
                    logger.info(f"找到 {len(scripts)} 个JSON脚本")

                    pricing_data = {
                        "nightly_price": 'NaN',
                        "cleaning_fee": 'NaN',
                        "service_fee": 'NaN',
                        "total": 'NaN'
                    }
                    
                    price_found = False
                    for script in scripts:
                        try:
                            data = json.loads(script.string)
                            if 'pdp_listing_booking_details' in str(data):
                                pricing_module = data.get('pdp_listing_booking_details', {})
                                pricing_data["nightly_price"] = pricing_module.get('price', {}).get('rate', {}).get('amount', 'NaN')
                                pricing_data["cleaning_fee"] = pricing_module.get('price', {}).get('cleaning_fee', {}).get('amount', 'NaN')
                                pricing_data["service_fee"] = pricing_module.get('price', {}).get('service_fee', {}).get('amount', 'NaN')
                                pricing_data["total"] = pricing_module.get('price', {}).get('total', {}).get('amount', 'NaN')
                                logger.info(f"价格信息: {pricing_data}")
                                break
                        except json.JSONDecodeError:
                            continue
                    if not price_found:
                        logger.warning("未找到价格信息")
                except Exception as e:
                    logger.error(f"提取定价数据时出错: {str(e)}")
                
                # 添加可用日期的完整数据
                df = pd.concat([df, pd.DataFrame({
                    'Host': [listing_url],
                    'Check-in Date': [checkin_date],
                    'Check-out Date': [checkout_date],
                    'Guest': [Guests],
                    'bedrooms': [bedrooms],
                    'beds': [beds],
                    'baths': [baths],
                    'years_hosting': [years_hosting],
                    'nightly_price': [pricing_data["nightly_price"]],
                    'cleaning_fee': [pricing_data["cleaning_fee"]],
                    'service_fee': [pricing_data["service_fee"]],
                    'total': [pricing_data["total"]],
                    'availability': ['Available']
                })], ignore_index=True)

                success_count += 1
                logger.info(f"成功抓取日期 {checkin_date} 的数据")
                
            except Exception as e:
                fail_count += 1
                logger.error(f"处理日期 {checkin_date} 时出错: {str(e)}")
                
            finally:
                current_date += timedelta(days=1)

        logger.info(f"\n抓取统计:")
        logger.info(f"总天数: {num_days}")
        logger.info(f"成功天数: {success_count}")
        logger.info(f"失败天数: {fail_count}")
        logger.info(f"成功率: {(success_count/num_days)*100:.2f}%")
                
    except Exception as e:
        logger.error(f"抓取过程出错: {str(e)}")
        
    finally:
        driver.quit()
        logger.info("浏览器已关闭")
        
    return df

### 单个网页抓去测试

In [42]:
#Example URL and parameters
listing_url = 'https://www.airbnb.co.nz/rooms/837352260137971048'
guests = 3
num_days = 10 #change to 180 for 6 Months

#Scrape pricing data for the next 180 days
pricing_data = scrape_listing_pricing(listing_url, guests, num_days)

pricing_data

2024-11-02 18:23:15,772 [INFO] 开始抓取房源: https://www.airbnb.co.nz/rooms/837352260137971048
2024-11-02 18:23:15,773 [INFO] 设置: guests=3, num_days=10
2024-11-02 18:23:15,774 [INFO] ====== WebDriver manager ======
2024-11-02 18:23:16,531 [INFO] Get LATEST chromedriver version for google-chrome
2024-11-02 18:23:16,786 [INFO] Get LATEST chromedriver version for google-chrome
2024-11-02 18:23:16,832 [INFO] Driver [C:\Users\jiann\.wdm\drivers\chromedriver\win64\130.0.6723.91\chromedriver-win32/chromedriver.exe] found in cache
2024-11-02 18:23:17,974 [INFO] Chrome WebDriver 初始化成功
2024-11-02 18:23:17,975 [INFO] 访问页面: https://www.airbnb.co.nz/rooms/837352260137971048
2024-11-02 18:23:25,250 [INFO] 页面解析完成
2024-11-02 18:23:25,262 [INFO] 找到 6 个房屋信息元素
2024-11-02 18:23:25,262 [INFO] 房屋信息: Guests=6.0, bedrooms=3.0, beds=4.0, baths=2.0, years_hosting=NaN
2024-11-02 18:23:25,263 [INFO] 
处理日期: 2024-11-02 到 2024-11-04
2024-11-02 18:23:29,325 [INFO] 找到 123 个日期单元格
2024-11-02 18:23:29,327 [INFO] 找到 7 个JSON脚本
2

,Check-in Date,Check-out Date,Host,Guest,bedrooms,beds,baths,years_hosting,nightly_price,cleaning_fee,service_fee,total,availability
0,2024-11-02,2024-11-04,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
1,2024-11-03,2024-11-05,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
2,2024-11-04,2024-11-06,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
3,2024-11-05,2024-11-07,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
4,2024-11-06,2024-11-08,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
5,2024-11-07,2024-11-09,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
6,2024-11-08,2024-11-10,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
7,2024-11-09,2024-11-11,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
8,2024-11-10,2024-11-12,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available
9,2024-11-11,2024-11-13,https://www.airbnb.co.nz/rooms/837352260137971048,6.0,3.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN,Available


### 自动多网页抓取测试
#### 网页选择器

In [9]:
import pandas as pd
urls = [
'https://www.airbnb.co.nz/rooms/49525122',
'https://www.airbnb.co.nz/rooms/49525472',
'https://www.airbnb.co.nz/rooms/716883705085875481',
'https://www.airbnb.co.nz/rooms/1044551981539410265',
'https://www.airbnb.co.nz/rooms/635947170438929163',
'https://www.airbnb.co.nz/rooms/1044551981539410265',
'https://www.airbnb.co.nz/rooms/668902191198356462',
'https://www.airbnb.co.nz/rooms/38854776',
'https://www.airbnb.co.nz/rooms/856174211238380149',
'https://www.airbnb.co.nz/rooms/676112692890991678',
'https://www.airbnb.co.nz/rooms/769112025086830655',
'https://www.airbnb.co.nz/rooms/668902191198356462',
'https://www.airbnb.co.nz/rooms/709166860198621755',
'https://www.airbnb.co.nz/rooms/818542484636231684',
'https://www.airbnb.co.nz/rooms/837352260137971048',
'https://www.airbnb.co.nz/rooms/54370426',
'https://www.airbnb.co.nz/rooms/18348257',
'https://www.airbnb.co.nz/rooms/648591357028974926',
'https://www.airbnb.co.nz/rooms/669791209041202721',
'https://www.airbnb.co.nz/rooms/976482042592323453',
'https://www.airbnb.co.nz/rooms/969823904189469776',
'https://www.airbnb.co.nz/rooms/814639112400706392',
'https://www.airbnb.co.nz/rooms/1039583273388527380',
    
]

# Create a DataFrame
df = pd.DataFrame(urls, columns=['URL'])

# Export to Excel
df.to_excel('urls.xlsx', index=False)

In [ ]:
import pandas as pd

# Read the Excel file
df = pd.read_excel('urls.xlsx')

# The number of guests and days
guests = 3       #选择入住人数
num_days = 180  #选择抓去的天数

# Loop over each URL
for listing_url in df['URL']:
    print(listing_url)

    pricing_data = scrape_listing_pricing(listing_url, guests, num_days)
    
pricing_data

In [ ]:
pricing_data

In [ ]:
pricing_data.to_excel('pricing_data.xlsx', index=False)